# USA Name Data - kaggle - test

### The code below pulls and summarizes the data from the USA Name Data (BigQuery Dataset) from kaggle.com

In [22]:
# ******************************************************************************
# Dependencies and Setup
# ******************************************************************************
import pandas as pd

from google.cloud import bigquery
client = bigquery.Client()


# ******************************************************************************
# Define the SQL Query to capture all the data including state, year and the 
# total number of births in the USA Name data (from kaggle)
# ******************************************************************************
sql = """
    SELECT state, year, SUM(number) as count
    FROM `bigquery-public-data.usa_names.usa_1910_current`
    GROUP BY year, state
    ORDER BY state, year ASC
"""

# ******************************************************************************
# Create a pandas dataframe that captures the result of running the SQL Query 
# above. There are a total of 5508 rows included in this dataframe. It includes 
# all 50 states with data from 1910 to 2017 for each state (where it exists).
# ******************************************************************************
usa_name_data_df = client.query(sql).to_dataframe()

# ******************************************************************************
# Print the number of rows in teh dataset
# ******************************************************************************
num_rows = usa_name_data_df.shape[0]
print(f"There are {num_rows} rows in this dataset")


There are 5508 rows in this dataset


In [24]:
# ******************************************************************************
# Print a sample of the data included in the dataset
# ******************************************************************************
usa_name_data_df.head()

,state,year,count
0,AK,1910,115
1,AK,1911,84
2,AK,1912,141
3,AK,1913,110
4,AK,1914,245
